# MIGHT example on Wise-1 cancer data

## Import libraries and local functions

In [1]:
# dependency libararies
import warnings
import pickle
import time
import numpy as np
import pandas as pd

# treeple functions
from treeple.ensemble import HonestForestClassifier
from treeple.tree import ObliqueDecisionTreeClassifier
from treeple.stats import build_oob_forest

warnings.filterwarnings("ignore")

In [2]:
# local function for processing cancer data files
def get_X_y(f, root="./data/", cohort=[], verbose=False):
    df = pd.read_csv(root + f)
    non_features = [
        "Run",
        "Sample",
        "Library",
        "Cancer Status",
        "Tumor type",
        "Stage",
        "Library volume (uL)",
        "Library Volume",
        "UIDs Used",
        "Experiment",
        "P7",
        "P7 Primer",
        "MAF",
    ]
    sample_ids = df["Sample"]
    # if sample is contains "Run" column, remove it
    for i, sample_id in enumerate(sample_ids):
        if "." in sample_id:
            sample_ids[i] = sample_id.split(".")[1]
    target = "Cancer Status"
    y = df[target]
    # convert the labels to 0 and 1
    y = y.replace("Healthy", 0)
    y = y.replace("Cancer", 1)
    # remove the non-feature columns if they exist
    for col in non_features:
        if col in df.columns:
            df = df.drop(col, axis=1)
    nan_cols = df.isnull().all(axis=0).to_numpy()
    # drop the columns with all nan values
    df = df.loc[:, ~nan_cols]
    # if cohort is not None, filter the samples
    if cohort is not None:
        # filter the rows with cohort1 samples
        X = df[sample_ids.isin(cohort)]
        y = y[sample_ids.isin(cohort)]
    else:
        X = df
    if "Wise" in f:
        # replace nans with zero
        X = X.fillna(0)
    # impute the nan values with the mean of the column
    X = X.fillna(X.mean(axis=0))
    # check if there are nan values
    # nan_rows = X.isnull().any(axis=1)
    nan_cols = X.isnull().all(axis=0)
    # remove the columns with all nan values
    X = X.loc[:, ~nan_cols]
    if verbose:
        if nan_cols.sum() > 0:
            print(f)
            print(f"nan_cols: {nan_cols.sum()}")
            print(f"X shape: {X.shape}, y shape: {y.shape}")
        else:
            print(f)
            print(f"X shape: {X.shape}, y shape: {y.shape}")
    # X = X.dropna()
    # y = y.drop(nan_rows.index)

    return X, y

## Process the Wise-1 cancer data

In [3]:
DIRECTORY = "./"
N_JOBS = 4
N_EST = 100 # 100_000

# collect sample data
sample_list_file = DIRECTORY + "AllSamples.MIGHT.Passed.samples.txt"
sample_list = pd.read_csv(sample_list_file, sep=" ", header=None)
sample_list.columns = ["library", "sample_id", "cohort"]

# get the sample ids for specific cohorts
cohort1 = sample_list[sample_list["cohort"] == "Cohort1"]["sample_id"]
cohort2 = sample_list[sample_list["cohort"] == "Cohort2"]["sample_id"]
PON = sample_list[sample_list["cohort"] == "PanelOfNormals"]["sample_id"]

In [4]:
# obtain the samples for cohort 1
X, y = get_X_y("WiseCondorX.Wise-1.csv.gz", root=DIRECTORY, cohort=cohort1)

# Save the processed data to a CSV file for use in the Yggdrasil notebook
processed_data = X.copy()
processed_data["Cancer Status"] = y
processed_data.to_csv("processed_wise1_data.csv", index=False)

In [5]:
X.shape

(352, 2523)

## Run axis-aligned MIGHT

In [41]:
est = HonestForestClassifier(
    n_estimators=N_EST,
    max_samples=1.6,
    max_features=0.3,
    bootstrap=True,
    stratify=True,
    n_jobs=N_JOBS,
    random_state=23,
    honest_prior="ignore",
    honest_method='prune',
)

# record start time
start_time = time.perf_counter()

# obtain tree level posteriors
fitted_est, tree_proba = build_oob_forest(est, X, y)

# calculate fitting time
end_time = time.perf_counter()
time_dif = end_time-start_time

print(time_dif)

1.2514145409986668


In [42]:
# obtain forest level posteriors
forest_proba = np.nanmean(tree_proba, axis=0)

# Benchmarks

In [ ]:
# dependency libararies
import warnings
import pickle
import time
import numpy as np
import pandas as pd

# treeple functions
from treeple.ensemble import HonestForestClassifier
from treeple.tree import ObliqueDecisionTreeClassifier
from treeple import ObliqueRandomForestClassifier

import numpy as np
import pandas as pd
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from treeple.stats import build_oob_forest

import numpy as np
import pandas as pd
import time

import ydf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import sys

warnings.filterwarnings("ignore")


random_seed = 42


def benchmark_alg(n_vals, d_vals, classifier, repeats=7, random_state=42, log_file_path=None):  # NEW: pass path
    """
    Benchmarks MIGHT's training & inference time across multiple (n, d) data sizes,
    repeating each (n,d) `repeats` times to compute mean & std dev.

    Returns a DataFrame with columns:
      ["n", "d",
       "train_time_mean", "train_time_std",
       "inference_time_mean", "inference_time_std",
       "accuracy_mean", "accuracy_std"]
    """
    np.random.seed(random_state)
    results = []

    for n in n_vals:
        for d in d_vals:
            train_times = []
            inference_times = []
            accuracies = []

            for _ in range(repeats):
                # 1) Generate random data
                X = np.random.randn(n, d)

                # TODO Important - Ariel: Testing whether a simple Fortran layout here would improve perf.
                X = np.asfortranarray(X)

                y = np.random.randint(2, size=n)

                # 2) Split
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.3, random_state=random_state, stratify=y
                )

                # 3) Convert to DataFrame. TODO Ariel - Disable this to avoid potentially switching back to C order
                # df_train = pd.DataFrame(X_train)
                df_train = X_train
                # df_test  = pd.DataFrame(X_test)
                df_test = X_test

                # 4) Build classifier
                # You could define this earlier, or globally, etc.
                est = classifier

                # 5) Training time
                t0 = time.perf_counter()
                fitted_est, tree_proba = build_oob_forest(est, df_train, y_train)
                t1 = time.perf_counter()
                train_times.append(t1 - t0)

                # 6) Inference time
                t2 = time.perf_counter()

                # Get predicted probabilities directly from the fitted ensemble
                y_proba_test = fitted_est.predict_proba(df_test)[:, 1]

                # Convert probabilities to binary predictions
                y_pred_test = (y_proba_test >= 0.5).astype(int)

                # Measure inference time
                t3 = time.perf_counter()
                inference_times.append(t3 - t2)

                # Compute accuracy
                acc = accuracy_score(y_test, y_pred_test)
                accuracies.append(acc)

                # 7) Accuracy
                acc = accuracy_score(y_test, y_pred_test)
                accuracies.append(acc)

            # Mean/std
            train_time_mean = np.mean(train_times)
            train_time_std  = np.std(train_times)
            inf_time_mean   = np.mean(inference_times)
            inf_time_std    = np.std(inference_times)
            acc_mean        = np.mean(accuracies)
            acc_std         = np.std(accuracies)

            msg = (
                f"\n(n={n}, d={d}):\n"
                f"  train_time mean={train_time_mean:.4f}s, std={train_time_std:.4f}\n"
                f"  inference_time mean={inf_time_mean:.4f}s, std={inf_time_std:.4f}\n"
                f"  accuracy mean={acc_mean:.3f}, std={acc_std:.3f}"
            )

            # Print to screen
            print(msg)

            # Write to file right away, no buffering
            if log_file_path:  # NEW
                with open(log_file_path, "a") as f:  # open in append mode
                    f.write(msg + "\n")
                    f.flush()  # force flush, ensures content is written

            results.append({
                "n": n,
                "d": d,
                "train_time_mean": train_time_mean,
                "train_time_std": train_time_std,
                "inference_time_mean": inf_time_mean,
                "inference_time_std": inf_time_std,
                "accuracy_mean": acc_mean,
                "accuracy_std": acc_std,
            })

    return pd.DataFrame(results)

## MIGHT

In [ ]:
honest_classifier = HonestForestClassifier(
    n_estimators=1000,
    max_samples=1.0, # Use all samples for bootstrapping. Use 1.0 instead of 1. 1 means literally 1 sample
    max_features=128, # Keep this static
    bootstrap=True,
    stratify=True,
    n_jobs=-1, # Use all resources
    random_state=42,
    honest_prior="ignore",
    honest_method='prune',
    tree_estimator=ObliqueDecisionTreeClassifier(),
    max_depth = None, # Fully grow trees
)

oblique_classifier = ObliqueRandomForestClassifier(
    n_estimators=1000,
    max_samples=1.0, # Use all samples for bootstrapping. Use 1.0 instead of 1. 1 means literally 1 sample
    # max_features=128, # Keep this static
    bootstrap=True,
    n_jobs=-1, # Use all resources
    random_state=42,
    max_depth = None, # Fully grow trees
    oob_score=False # Don't Inference OOB samples
)

### Honest

In [ ]:
n_values = [128, 256,512,1024,2048,4096,8192, 16384, 32768, 65536, 131072]
d_values = [128,256,512,1024,2048,4096,8192, 16384, 32768, 65536, 131072]

# If you want a header first, you can do that outside:
with open("might_benchmark_output.txt", "w") as f:
    f.write("=== MIGHT Benchmark Output ===\n")
    f.flush()

df_bench = benchmark_alg(n_values, d_values, classifier=honest_classifier, repeats=3,
                            random_state=42, log_file_path="might_benchmark_output.txt")

# Final summary
summary = "\n=== MIGHT Benchmark Results (Averaged Over Repeats) ===\n"
print(summary)
with open("might_benchmark_output.txt", "a") as f:
    f.write(summary + "\n")
    f.flush()

print(df_bench)
with open("might_benchmark_output.txt", "a") as f:
    f.write(df_bench.to_string(index=False) + "\n")
    f.flush()

### Oblique, non-Honest

In [ ]:
n_values = [128, 256,512,1024,2048,4096,8192, 16384, 32768, 65536]#, 131072]
d_values = [128,256,512,1024,2048,4096,8192, 16384, 32768, 65536]#, 131072]

perf_logpath = "../benchmark_results/oblique_treeple_benchmark_output.txt"

# If you want a header first, you can do that outside:
with open(perf_logpath, "w") as f:
    f.write("=== OBLIQUE MIGHT Benchmark Output ===\n")
    f.flush()

df_bench = benchmark_alg(n_values, d_values, classifier=oblique_classifier, repeats=3,
                            random_state=42, log_file_path=perf_logpath)

# Final summary
summary = "\n=== MIGHT Benchmark Results (Averaged Over Repeats) ===\n"
print(summary)
with open(perf_logpath, "a") as f:
    f.write(summary + "\n")
    f.flush()

print(df_bench)
with open(perf_logpath, "a") as f:
    f.write(df_bench.to_string(index=False) + "\n")
    f.flush()


(n=128, d=128):
  train_time mean=1.8768s, std=0.0338
  inference_time mean=0.0986s, std=0.0013
  accuracy mean=0.444, std=0.032

(n=128, d=256):
  train_time mean=1.8829s, std=0.0079
  inference_time mean=0.1003s, std=0.0013
  accuracy mean=0.496, std=0.024

(n=128, d=512):
  train_time mean=1.9386s, std=0.0532
  inference_time mean=0.1047s, std=0.0014
  accuracy mean=0.573, std=0.024

(n=128, d=1024):
  train_time mean=2.0406s, std=0.0244
  inference_time mean=0.0981s, std=0.0011
  accuracy mean=0.581, std=0.053

(n=128, d=2048):
  train_time mean=2.2252s, std=0.0386
  inference_time mean=0.1008s, std=0.0006
  accuracy mean=0.479, std=0.079

(n=128, d=4096):
  train_time mean=3.7329s, std=0.0957
  inference_time mean=0.1124s, std=0.0109
  accuracy mean=0.453, std=0.044


## YDF 🌳

### Synthetic Scaling Data

In [ ]:
ydf_est = ydf.RandomForestLearner(
                    label="label",
                    random_seed=random_seed,
                    split_axis="SPARSE_OBLIQUE",
                    num_trees=1000,
                    bootstrap_training_dataset=True,
                    bootstrap_size_ratio=1.0,
                    num_threads=96,
                    max_depth=-1,

                    sparse_oblique_projection_density_factor=128, # This should finally set n_nonzeros in proj matrix constant (in expectation)

                    # sparse_oblique_max_num_projections = 128,
                    # sparse_oblique_num_projections_exponent=0.0, # """ This should make max_num_projections always WIN """

                    # ''' Should skip computing OOB after training'''
                    honest=False,
                    compute_oob_performances=False, # Should skip computing OOB after training
                )

In [ ]:
n_values = [128, 256,512,1024,2048,4096,8192, 16384, 32768, 65536]#, 131072]
p_values = [128,256,512,1024,2048,4096,8192, 16384, 32768, 65536]#, 131072]

perf_logpath = "../results/ydf_benchmark_output.txt"

# If you want a header first, you can do that outside:
with open(perf_logpath, "w") as f:
    f.write("=== YDF Benchmark Output ===\n")
    f.flush()

df_bench = benchmark_alg(n_values, p_values, repeats=3, classifier=ydf_est,
                            random_state=42, log_file_path=perf_logpath)

summary = "\n=== YDF Benchmark Results (Averaged Over Repeats) ===\n"
print(summary)
with open(perf_logpath, "a") as f:
    f.write(summary + "\n")
    f.flush()

print(df_bench)
with open(perf_logpath, "a") as f:
    f.write(df_bench.to_string(index=False) + "\n")
    f.flush()

### Real MIGHT data

In [ ]:
###############################################
# 1) Imports and Basic Setup
###############################################
import time
import pandas as pd
import numpy as np

# This is the current Python package for Yggdrasil Decision Forests.
import ydf

###############################################
# 2) Load the Exact Same Processed Data
###############################################
# In your MIGHT notebook, you saved something like "processed_wise1_data.csv".
# Let's read that in so YDF sees the identical data.

PROCESSED_DATA = "processed_wise1_data.csv"
df = pd.read_csv(PROCESSED_DATA)

# Suppose the label column is named "Cancer Status".
LABEL_COL = "Cancer Status"

print("DataFrame shape:", df.shape)
# print("Columns:", df.columns.tolist())

###############################################
# 3) Verify We Have the Same Classification Task
###############################################
# Ensure that the label distribution matches what you see in MIGHT (e.g., y.value_counts()).

num_rows, num_cols = df.shape
if LABEL_COL not in df.columns:
    raise ValueError(f"Label column {LABEL_COL!r} not found in CSV columns!")

print("Number of samples (rows):", num_rows)
print("Number of columns:", num_cols)
print(f"Label '{LABEL_COL}' distribution:\n{df[LABEL_COL].value_counts()}")

# Optional: Print the first few columns to confirm
print("Feature columns (excluding label):")
feature_cols = [c for c in df.columns if c != LABEL_COL]
print(feature_cols[:10], "..." if len(feature_cols) > 10 else "")

###############################################
# 4) Train a YDF Random Forest
###############################################
# Provide the label name, and optionally set random_seed for reproducibility.

rf_learner = ydf.RandomForestLearner(
    label=LABEL_COL,
    random_seed=42,  # ensures reproducible random sampling
    num_trees=100,   # match MIGHT for fair comparison
    max_depth=10     # or whatever matches your MIGHT hyperparams
)

start_time = time.perf_counter()
rf_model = rf_learner.train(df)
end_time = time.perf_counter()

train_time = end_time - start_time
print(f"\n[INFO] Yggdrasil RF trained in {train_time:.2f} seconds.")

###############################################
# 5) Evaluate / Inspect the Model
###############################################
# Evaluate on the same data (for quick demonstration).
evaluation = rf_model.evaluate(df)
print("\nEvaluation metrics on the training set:")
print(evaluation)

# If you want, you can also get predictions or partial-dependence analysis:
predictions = rf_model.predict(df)
print("\nSample predictions (first 5 rows):")
print(predictions)

###############################################
# 6) Confirm Similarity with MIGHT
###############################################
# For a fair side-by-side timing:
# - MIGHT uses the same "processed_wise1_data.csv".
# - Both have 100 trees, same random seed if possible.
# - Start/stop the timer at the exact training call.
#
# Now the only difference should be the algorithms/impl details themselves.
###############################################

In [ ]:
import pandas as pd
import numpy as np
import time

import ydf  # Yggdrasil Decision Forests
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1) Read the same processed CSV as MIGHT
CSV_FILE = "processed_wise1_data.csv"
df = pd.read_csv(CSV_FILE)

LABEL_COL = "Cancer Status"
if LABEL_COL not in df.columns:
    raise ValueError(f"Missing label column {LABEL_COL!r} in CSV.")

# 2) Train/test split (hold-out)
#    We'll separate 30% of the data for testing. Use the same random_state so you can replicate in MIGHT.
X = df.drop(columns=[LABEL_COL])
y = df[LABEL_COL]
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=y)

print("Train set shape:", train_df.shape)
print("Test set shape:", test_df.shape)

# 3) Build a YDF RandomForest with ~the same hyperparams as MIGHT
#    e.g. 100 trees, random_seed=23 or 42, etc.
rf_learner = ydf.RandomForestLearner(
    label=LABEL_COL,
    random_seed=42,
    num_trees=100 #100000
)

# 4) Train on the HOLD-OUT train set
start_time = time.perf_counter()
rf_model = rf_learner.train(train_df)  # Only pass the train portion
end_time = time.perf_counter()
train_time = end_time - start_time
print(f"\nYDF model trained in {train_time:.2f} seconds on {len(train_df)} examples.")

# 5) Evaluate on the test set using YDF's built-in evaluate()
evaluation = rf_model.evaluate(test_df)
print("\n=== YDF Evaluate() on Test Set ===")
print(evaluation)

#   By default, it prints metrics like "accuracy", "confusion matrix", "ROC AUC",
#   etc. in a structured text output.

# 6) (Optional) Compute scikit-learn metrics on the test set
#    The YDF model's `.predict()` returns a DataFrame with probabilities for label=1 by default.
preds = rf_model.predict(test_df)  # shape: (num_test_examples,) containing probabilities
preds_np = preds#.to_numpy().ravel()

# Convert probabilities -> predicted class using 0.5 threshold
pred_labels = (preds_np >= 0.5).astype(int)

test_y = test_df[LABEL_COL].values

acc = accuracy_score(test_y, pred_labels)
cm = confusion_matrix(test_y, pred_labels)
cls_rpt = classification_report(test_y, pred_labels)

print("\n=== Scikit-learn metrics on Test Set ===")
print(f"Accuracy: {acc:.3f}")
print("Confusion matrix:\n", cm)
print("\nClassification Report:\n", cls_rpt)

# 7) Display sample predicted probabilities (first 10), to mimic the style you saw
print("\nSample predicted probabilities (first 10):")
print(preds_np[:10])